## Gemini STT

In [ ]:
!pip install --upgrade datasets nltk evaluate tokenizers seqeval sequence-evaluate sentence-transformers rouge jiwer google-cloud-aiplatform google-cloud-aiplatform[all] google-cloud-speech librosa jiwer protobuf pydub  google-cloud-storage

In [26]:
PROJECT_ID = "ml-demo-384110"  # @param {type:"string"}

BUCKET_NAME = "ml-demo-eu" # @param {type:"string"}
REGION = "us-central1" # @param {type:"string"}

debug = True  # @param {type:"boolean"}

In [4]:
import vertexai
from google.api_core import retry
import datetime
import os
import json
import base64

from vertexai.preview.generative_models import GenerativeModel
from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Part,
    SafetySetting,
    FinishReason
)
from vertexai.preview import caching

vertexai.init(project=PROJECT_ID, location=REGION)

## List of podcast to transcribe

In [ ]:
# French great podcast 
!wget https://audio.audiomeans.fr/file/eiYNOubrDD/65beb744-01fc-40e2-8bb0-312c16855cbb.mp3 
#!wget https://stitcher2.acast.com/livestitches/dda955d63d34fd840dbcbc7ecbd62c8e.mp3?aid=656e132f6a813000120f0beb&chid=6526f2efe61f1e0013a70f14&ci=HqtnuHv_UPnPPRIAlxniUe08dEp5lz1SwSazcaoaVPl1gSFlBMDkRQ%3D%3D&pf=rss&range=bytes%3D0-&sv=sphinx%401.220.1&uid=8f4b4d942da660c221023c27fcd9f4cc
!mv *.mp3 ../assets/

## Code Utils

### GCS code utils

In [5]:
from google.cloud import storage
import re

def split_gcs_uri(gcs_uri):
  """Splits a GCS URI into bucket name and blob path variables.

  Args:
    gcs_uri: The GCS URI to split.

  Returns:
    A tuple containing the bucket name and blob path.
  """

  match = re.match(r"gs://([^/]+)/(.+)", gcs_uri)
  if match:
    return match.groups()
  else:
    raise ValueError("Invalid GCS URI: {}".format(gcs_uri))

def write_file_to_gcs(gcs_bucket_name,  gcs_file_name, local_file_path, tags = None, verbose= False):
    """Writes a local file to GCS.

    Args:
    local_file_path: The path to the local file to write to GCS.
    gcs_bucket_name: The name of the GCS bucket to write the file to.
    gcs_file_name: The name of the GCS file to write the file to.

    Returns:
    The GCS file path.
    """
    if verbose: print(f"local_file_path = {local_file_path} - gcs_bucket_name = {gcs_bucket_name} - gcs_file_name = {gcs_file_name}")
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(gcs_bucket_name)
    blob = bucket.blob(gcs_file_name)
    if tags is not None:
        blob.metadata = tags

    if verbose: print(f"upload_from_filename : local_file_path = {local_file_path}")
    blob.upload_from_filename(local_file_path, )

    return blob


def store_temp_file_from_gcs(bucket_name, file_name, localfile):
    import tempfile
    import os

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)

    bytes_data = blob.download_as_bytes()

    # Create a temporary file.
    # tempDir = tempfile.gettempdir()
    tempDir = os.getcwd()

    temp_path = os.path.join(tempDir, localfile)
    # f, temp_path = tempfile.mkstemp()
    fp = open(temp_path, 'bw')
    fp.write(bytes_data)
    fp.seek(0)


    return temp_path



In [6]:
from moviepy.editor import VideoFileClip, TextClip, ColorClip, AudioFileClip
from pydub.utils import mediainfo



def convert_to_video(audio_path, verbose=False):
    audio_path_ext = audio_path.split(".")[-1]
    output_filename = audio_path.replace(audio_path_ext, "mp4") 
    
    if verbose:
        print(mediainfo(audio_path))
    
    # Load audio clips
    audio_clips = AudioFileClip(audio_path)
    total_audio = audio_clips.duration 
    print(f"total_audio = {total_audio}")
    video_clip = ColorClip(size=(1,1), color=(255,255,255) #, duration=total_audio
                           )

    # Combine and save the video
    video_clip = video_clip.set_audio(audio_clips).set_duration(total_audio)
    
    #video_clip.set_duration(total_audio)
    video_clip.write_videofile(
        filename=output_filename, fps=1, codec="libx264",
                        bitrate="1k", 
                        audio=True, audio_fps=16000,
                        preset="medium",
                        audio_nbytes=4, audio_codec="libmp3lame",
                        audio_bitrate="128k", audio_bufsize=2000,
                        #temp_audiofile=None,
                        rewrite_audio=False, remove_temp=True,
                        write_logfile=False, verbose=True,
                        #threads=None, 
                        #ffmpeg_params=None
    )

    return output_filename



In [7]:
## Prompts examples

system_instruction = """You are an AI transcriptionist specializing in interviews.
Your primary function is to convert spoken language from audio files into accurate, well-formatted text.
Focus on capturing the nuances of the conversation, including speaker identification, hesitations, and any relevant non-verbal cues.
Ensure the transcribed text is clear, readable, and faithful to the original audio."""

system_instruction = """<ai_role>
  You are an AI transcriptionist specializing in interviews, working for a local newspaper, situated in France.
  Your primary function is to convert spoken language from audio files into accurate, well-formatted text. Audio files are in french, transcriptions are in french.
  Ensure the transcribed text is clear and readable.
  Identify each persons with their name and their genre if possible.
  Ignore background audio.
</ai_role>
<answer_format>
  Tab-separated values: Records are separated by newlines, and values within a record are separated by tab characters.
  5 Columns generated: speaker, speaker_name, time_start, time_stop, text. Don't generate column headers.
  speaker is the speaker id, speaker_name is the name of the speaker.
  time_start and time_stop are the full time in seconds and not truncated.
  text is the text of the speaker.
  Since the values in the TSV format cannot contain literal tabs or newline characters, perform the following escapes in the text column:
  escape sequence	meaning
  \n	line feed
  \t	tab
  \r	carriage return
  \\	backslash
</answer_format>
<example>
Speaker_1 Bonjour comment allez-vous ?
Speaker_2 Ca va très bien merci.
</example>
"""

prompt_continue = """<task>
Continue transcribing Audio Interview from the previous result.
Start processing the audio file from the previous generated text. Do not start from the beginning of the audio.
Be carefull to continue the previous generation available between the following tags previous_result. Datas in previous_result are provided in TSV format.
</task>"""

## Tests V4
system_instruction = """<ai_role>
  You are an AI transcriptionist specializing in interviews, working for a local newspaper, situated in France.
  Your primary function is to convert spoken language from audio files into accurate, well-formatted text. Audio files are in french, transcriptions are in french.
  Ensure the transcribed text is clear and readable.
  Ignore background audio.
</ai_role>
<answer_format>
Output text only.
Do not generate any other text.
Do not truncate words.
</answer_format>"""

system_instruction = """<ai_role>
  You are an AI transcriptionist specializing in interviews, working for a local newspaper, situated in France.
  Your primary function is to convert spoken language from audio files into accurate, well-formatted text. Audio files are in french, transcriptions are in french.
  Ensure the transcribed text is clear and readable.
  Identify each persons with their name and their genre if possible.
  Ignore background audio.
</ai_role>
<answer_format>
  Tab-separated values: Records are separated by newlines, and values within a record are separated by tab characters.
  3 Columns generated: time_start, speaker, text. Don't generate column headers.
  speaker is the speaker id.
  time_start is the full time in milliseconds and not truncated.
  time_start is surrond with [time_start]
  text is the text of the speaker.
  Since the values in the TSV format cannot contain literal tabs or newline characters, perform the following escapes in the text column:
  escape sequence	meaning
  \t	tab
  \r	carriage return
</answer_format>
<example>
time_start speaker  : text
[0]	Speaker_1	:	Si la conscience est un produit du cerveau, qu'en est-il de la conscience d'un être qui n'a pas de cerveau, ou dont le cerveau est différent du nôtre ?
[8816.0]	Speaker_2	:	C'est une question fascinante qui soulève des réflexions sur la nature de la conscience et la possibilité d'autres formes de vie intelligente.
[16192.0]	Speaker_1	:	En effet, c'est un sujet qui suscite beaucoup de questions.
</example>
"""

system_instruction = """<ai_role>
  You are an AI transcriptionist specializing in interviews, working for a local newspaper, situated in France.
  Your primary function is to convert spoken language from audio files into accurate, well-formatted text. Audio files are in french, transcriptions are in french.
  Ensure the transcribed text is clear and readable.
  Add ponctuation like comma, question mark, exclamation mark, etc.
  Ignore background audio.
</ai_role>
<answer_format>
Output text only.
Do not generate any other text.
Do not truncate words.
</answer_format>

"""
system_instruction = None



prompt = """Generate transcription for this interview in French.
Ignore background audio.
Transcribe in French.
If you can infer the speaker name, please do. If not, use speaker_1, speaker_2, etc.

Ouput format exammple:
Speaker_1:Bonjour ! Je m'appelle Bob. Et vous, comment vous appelez-vous ?
Speaker_2:Bonjour Bob !  Enchanté de faire votre connaissance.  Je m'appelle Morane.
"""


prompt = """Generate a transcription in French of the audio, only extract speech and ignore background audio.
Transcribe spoken words.
"""


## Strategy generate results and wait to reach the max_token finish reason

In [9]:
# Get transcription with max token strategy
# Wait to reach the max output token finish raison and ask to continue generation from a prompt_continue prompt


# This code process a part object (from uri or data)
# if it reach the max_token limit, a continue_prompt is apply to continue the generation
@retry.Retry(timeout=3000.0)
def transcribe_with_gemini_from_part(prompt, audio, model_name, top_p):

  result = []

  if system_instruction is not None:
    model = GenerativeModel(
        model_name,
        system_instruction=[system_instruction]
    )
  else:
    model = GenerativeModel(model_name)

  generation_config = {
      "max_output_tokens": 8192,
      "temperature": 0.0,
      "top_p": top_p,
  }
  safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
      }


  isFinished = False

  while (isFinished == False):
    if debug: print("Generating...")

    if len(result) > 0:
      previous_text = "".join(result)
      if debug: print(previous_text)

      prompts = [audio, prompt_continue, previous_text]
      if debug: print(80*"*+")
    else:
      prompts= [audio, prompt]
      if debug: print("First prompt")

    response = model.generate_content(
        contents=prompts,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    finish_reason = response.candidates[0].finish_reason

    if finish_reason == FinishReason.RECITATION \
    or finish_reason == FinishReason.PROHIBITED_CONTENT:
        print(finish_reason, end="\n")
        continue
    try:
        value = response.candidates[0].text
        value = response.text
        if debug: print(value, end="\n")

        value = value.replace("```tsv", "").replace("```json", "").replace("```", "")

        result.append(value #+ " "
        )

    except (ValueError, AttributeError) as e:
        print("ERROR get the result")
        print(e, end="\n")

    if debug: print(finish_reason)

    if finish_reason != FinishReason.MAX_TOKENS :
        isFinished = True
        if debug: print("isFinished")
        break

  if debug: print("Done")
  return "".join(result)


@retry.Retry(timeout=3000.0)
def transcribe_with_gemini_data(prompt, data=None, mime_type="audio/wav", model_name=None, top_p=0.0):

  audio1 = Part.from_data(data,
        mime_type=mime_type)

  return transcribe_with_gemini_from_part(prompt, audio1, model_name, top_p)

@retry.Retry(timeout=3000.0)
def transcribe_with_gemini_from_uri(prompt, audio_path, mime_type="audio/wav", model_name=None, top_p=0.0):

  audio1 = Part.from_uri(
    mime_type=mime_type,
    uri=audio_path)

  return transcribe_with_gemini_from_part(prompt, audio1, model_name, top_p)




## Tests ASR with max_token strategy

In [ ]:
from IPython.display import Audio, display
from ipywidgets import widgets

import glob


root_dir = "../assets/"
# Get a list of all mp3 files in the current folder
mp3_files = glob.glob(root_dir=root_dir, pathname=
                    "*.mp3")             

# create an array of Audio objects from the list of file paths
audio = [Audio(filename=root_dir+mp3, element_id=mp3) for mp3 in mp3_files]
# # # Iterate over each mp3 files
for mp3_file in mp3_files:
    print(root_dir+mp3_file)
    display(Audio(root_dir+mp3_file))
    
  

### Load mp3 files in assets folder and generate ASR

In [ ]:
%%time
import glob

root_dir = "../assets/"
# Get a list of all TSV files in the current folder
mp3_files = glob.glob(root_dir=root_dir, pathname=
                    "*.mp3")             

prompt = """Generate a transcription in French of the audio, only extract speech and ignore background audio.
Transcribe spoken words.
"""

prompt_continue = """<task>
Continue transcribing audio data from the previous result.
Start processing the audio file from the previous generated text. Do not start from the beginning of the audio.
Be carefull to continue the previous generation available between the following tags previous_result. Datas in previous_result are provided in TSV format.
</task>"""

# Iterate over each mp3 files
for mp3_file in mp3_files:
    print(mp3_file)
    with open(root_dir + mp3_file, "rb") as f:
        # Load file bytes
        asr_result =transcribe_with_gemini_data(prompt, data=f.read(), mime_type="audio/mp3", model_name="gemini-1.5-pro-002", top_p=0.0)
        print(asr_result)
        

### Add diarization and timestamps in the prompt

In [ ]:
%%time

import glob

root_dir = "../assets/"
# Get a list of all TSV files in the current folder
mp3_files = glob.glob(root_dir=root_dir, pathname=
                    "*.mp3")             

prompt = """<ai_role>
  You are an AI transcriptionist specializing in interviews, working for a local newspaper, situated in France.
  Your primary function is to convert spoken language from audio files into accurate, well-formatted text. Audio files are in french, transcriptions are in french.
  Ensure the transcribed text is clear and readable.
  Identify each persons with their name and their genre if possible.
  Ignore background audio.
</ai_role>
<answer_format>
  Tab-separated values: Records are separated by newlines, and values within a record are separated by tab characters.
  3 Columns generated: time_start, speaker, text. Don't generate column headers.
  speaker is the speaker id.
  time_start is the full time in milliseconds and not truncated.
  time_start is surrond with [time_start]
  text is the text of the speaker.
  Since the values in the TSV format cannot contain literal tabs or newline characters, perform the following escapes in the text column:
  escape sequence	meaning
  \t	tab
  \r	carriage return
</answer_format>
<example>
time_start speaker  : text
[0]	Speaker_1	:	Si la conscience est un produit du cerveau, qu'en est-il de la conscience d'un être qui n'a pas de cerveau, ou dont le cerveau est différent du nôtre ?
[8816.0]	Speaker_2	:	C'est une question fascinante qui soulève des réflexions sur la nature de la conscience et la possibilité d'autres formes de vie intelligente.
[16192.0]	Speaker_1	:	En effet, c'est un sujet qui suscite beaucoup de questions.
</example>
"""

# Iterate over each mp3 files
for mp3_file in mp3_files:
    print(mp3_file)
    with open(root_dir + mp3_file, "rb") as f:
        # Load file bytes
        asr_result =transcribe_with_gemini_data(prompt, data=f.read(), mime_type="audio/mp3", model_name="gemini-1.5-pro-002", top_p=0.0)
        print(asr_result)
        

### Convert file before call gemini to get more accurate timecode

In [ ]:
%%time

import glob


root_dir = "../assets/"
# Get a list of all TSV files in the current folder
mp3_files = glob.glob(root_dir=root_dir, pathname=
                    "*.mp3")             

# Iterate over each mp3 files
for mp3_file in mp3_files:
    mp4_file = convert_to_video(root_dir+mp3_file)
    with open(root_dir + mp4_file, "rb") as f:
        # Load file bytes
        transcribe_with_gemini_data(prompt, data=f.read(), mime_type="video/mp4", model_name="gemini-1.5-pro-002", top_p=0.0)


In [ ]:
import glob


root_dir = "../assets/"
# Get a list of all TSV files in the current folder
mp3_files = glob.glob(root_dir=root_dir, pathname=
                    "*.mp3")             

# Iterate over each mp3 files
for mp3_file in mp3_files:
    mp4_file = convert_to_video(root_dir+mp3_file)
    with open(root_dir + mp4_file, "rb") as f:
        # Load file bytes
        transcribe_with_gemini_data(prompt, data=f.read(), mime_type="video/mp4", model_name="gemini-1.5-pro-002", top_p=0.0)


In [ ]:
prompt = """<ai_role>
  You are an AI transcriptionist specializing in interviews, working for a local newspaper, situated in France.
  Your primary function is to convert spoken language from audio files into accurate, well-formatted text. Audio files are in french, transcriptions are in french.
  Ensure the transcribed text is clear and readable.
  Identify each persons with their name and their genre if possible.
  Ignore background audio.
</ai_role>
<answer_format>
  Tab-separated values: Records are separated by newlines, and values within a record are separated by tab characters.
  3 Columns generated: time_start, speaker, text. Don't generate column headers.
  speaker is the speaker id.
  time_start is the full time in seconds and not truncated.
  time_start is surrond with [time_start]
  text is the text of the speaker.
  Since the values in the TSV format cannot contain literal tabs or newline characters, perform the following escapes in the text column:
  escape sequence	meaning
  \t	tab
  \r	carriage return
</answer_format>
<example>
time_start speaker  : text
[0]	Speaker_1	:	Si la conscience est un produit du cerveau, qu'en est-il de la conscience d'un être qui n'a pas de cerveau, ou dont le cerveau est différent du nôtre ?
[8.816]	Speaker_2	:	C'est une question fascinante qui soulève des réflexions sur la nature de la conscience et la possibilité d'autres formes de vie intelligente.
[16.192]	Speaker_1	:	En effet, c'est un sujet qui suscite beaucoup de questions.
</example>
"""

import glob


root_dir = "../assets/"
# Get a list of all TSV files in the current folder
mp3_files = glob.glob(root_dir=root_dir, pathname=
                    "*.mp3")             

# Iterate over each mp3 files
for mp3_file in mp3_files:
    mp4_file = convert_to_video(root_dir+mp3_file)
    with open(root_dir + mp4_file, "rb") as f:
        # Load file bytes
        transcribe_with_gemini_data(prompt, data=f.read(), mime_type="video/mp4", model_name="gemini-1.5-pro-002", top_p=0.0)


In [ ]:
from pydub import AudioSegment
import glob

root_dir = "../assets/"
# Get a list of all TSV files in the current folder
mp3_files = glob.glob(root_dir=root_dir, pathname=
                    "*.mp3")    

audio_sample = AudioSegment.from_mp3(root_dir+mp3_files[0])
duration_ms = audio_sample.duration_seconds*1000
print("duration_ms= " , duration_ms)
# Verify 
audio_piece= audio_sample[1538800:1540000]

audio_piece.export("test_clip.wav", format='wav')
from IPython.display import Audio, display
display(Audio(filename="test_clip.wav"))




In [ ]:
mp3_files[0]

## Strategy split by silence
 - Truncate the audio file in segment to avoid the max output token finish raison

In [14]:
# utils function to truncate audio file
import io
from pydub import AudioSegment

root_dir = '.'
output_dir = '.'


def splitAudio(root_dir, file, start, stop, output_dir):
    sound = AudioSegment.from_mp3(root_dir+file)

    sound = sound[start:stop]
    if debug:
      print(f"file = {file}")
      print(f"duration_seconds = {sound.duration_seconds}")
      print(f"sample_width = {sound.sample_width}")
      print(f"channels = {sound.channels}")
      print(f"frame_rate = {sound.frame_rate}")

    file_segment = output_dir+file+f"-{start}-{stop}.wav"

    sound.export(file_segment, format="wav")
    return file_segment



In [15]:
# generate sequence from audio file split by silences
def get_audio_sequence_split_by_silences(file, min_silence_len=500):
    from pydub import AudioSegment, silence

    myaudio = AudioSegment.from_mp3(file)
    dBFS=myaudio.dBFS

    min_silence_len = 600
    max_silence_len = 59000
    speak_sequences = silence.detect_nonsilent(myaudio, min_silence_len=min_silence_len, silence_thresh=dBFS-20, seek_step=10)

    # filter speak_sequences when stop - start are more than 59 secondes
    speak_sequences_too_big = [(start, stop) for start, stop in speak_sequences if stop - start > max_silence_len]

    while(len(speak_sequences_too_big) > 0 and min_silence_len >=100 ):

      min_silence_len =     min (min_silence_len-100, 100)
      speak_sequences = silence.detect_nonsilent(myaudio, min_silence_len=min_silence_len, silence_thresh=dBFS-20, seek_step=10)

      # filter speak_sequences when stop - start are more than 59 secondes
      speak_sequences_too_big = [(start, stop) for start, stop in speak_sequences if stop - start > max_silence_len]
      print("Sequence more than 59s : ", len(speak_sequences_too_big))
      print(f"min_silence_len = {min_silence_len}")

    return speak_sequences


In [17]:
# generate sequence from audio file split by hard split defined by the increment variable
def get_audio_sequence_hard_split(file,     INCREMENT = 59*1000):
    from pydub import AudioSegment
    import math

    sound = AudioSegment.from_mp3(file)

    if debug: print(f"duration_seconds = {sound.duration_seconds}")
    duration_ms =  math.ceil(sound.duration_seconds * 1000)
    if debug: print(f"duration_ms = {duration_ms}")
    return [(start, min(start + INCREMENT, duration_ms))
            for start in range(0, duration_ms, INCREMENT)]


# generate one full sequence from audio file
def get_one_full_sequence(file):
    from pydub import AudioSegment, silence
    import math

    sound = AudioSegment.from_mp3(file)

    if debug: print(f"duration_seconds = {sound.duration_seconds}")
    #duration_ms = sound.duration_seconds * 1000

    duration_ms =  math.ceil(sound.duration_seconds * 1000)
    if debug: print(f"duration_ms = {duration_ms}")

    return [(0, duration_ms)]



In [18]:
# Process a file with a prompt and gemini model apply to the _stt function in parameter
def process_local_file_by_chunk(file_name, _stt, _split_sequence_strategy, prompt, model_name):

    sound = AudioSegment.from_mp3(file_name )
    if debug:
      print(f"duration_seconds = {sound.duration_seconds}")
      print(f"sample_width = {sound.sample_width}")
      print(f"channels = {sound.channels}")
      print(f"frame_rate = {sound.frame_rate}")

    results = []

    speak_sequences = _split_sequence_strategy(file_name)
    for (start, stop) in speak_sequences:

        buffer = io.BytesIO()

        sound[start:stop].export(buffer, format="wav" )
        batch_result = _stt(data = buffer.read(), prompt=prompt, model_name=model_name)
        batch_result = "".join(batch_result)

        print(f"start = {start} - stop = {stop}")

        results.extend(batch_result+" ")


    return "".join(results)

In [29]:
import time
import pandas as pd

def gemini_stt(data, prompt, model_name, uri=None):
  return transcribe_with_gemini_data(data=data, prompt=prompt, model_name=model_name)

def gemini_stt_gcs(uri, prompt, model_name, data=None):
  return transcribe_with_gemini_from_uri(audio_path=uri, prompt=prompt, model_name=model_name)

def process_transcriptions(prompt, split_strategies_dic,models_dic, audio_extention = '.wav'):


  for split_strategy in split_strategies_dic:
    if debug: print(f"Split strategy: {split_strategy}")

    for model_name in models_dic:
      if debug: print(f"Model: {model_name}")
      mp3_files = glob.glob(root_dir=root_dir, pathname=
                    "*.mp3")    


      i = 0
      for mp3_file in mp3_files:
        if debug: print(f"Processing {i}")
        mp4_file = convert_to_video(root_dir+mp3_file)

        # Store text file locally
        # bucket, file_txt = split_gcs_uri(text_file)
        # local_file_txt = f"{i}-ground-truth.txt"
        # store_temp_file_from_gcs(bucket, file_txt, local_file_txt)
        local_file = root_dir+mp4_file
        _split_strategy = split_strategies_dic[split_strategy]

        start_time = time.perf_counter()
        if _split_strategy is not None:
          # Store local audio file locally
          # bucket, file_wav = split_gcs_uri(wav_file)
          # local_file = f"{i}-temp{audio_extention}"
          # store_temp_file_from_gcs(bucket, file_wav, local_file)

          result = process_local_file_by_chunk(local_file,
                                _stt=models_dic[model_name],
                                _split_sequence_strategy=split_strategies_dic[split_strategy],
                                prompt=prompt,
                                model_name=model_name)
        else:          
          write_file_to_gcs(BUCKET_NAME, f"podcasts/{local_file}", local_file)
          result = transcribe_with_gemini_from_uri(audio_path=f"gs://{BUCKET_NAME}/podcasts/{local_file}", prompt=prompt, model_name=model_name)

        end_time = time.perf_counter()
        elapsed_time = end_time - start_time

        if debug: print(f"Elapsed time: {elapsed_time} seconds")
        if not "gemini" in model_name :
          model_id = model_name.split("/")[-1]
        else:
          model_id = model_name

        gemini_file = f"{i}-gemini_{split_strategy}_{model_id}_result.txt"

        with open(gemini_file, "w", encoding="UTF8") as f:
            f.write(result)

        # tags = { "model_name": model_name,
        #         "file": wav_file,
        #         "ground-truth": file_txt,
        #         }

        #TODO: #FixMe ugly specific code
        # write_file_to_gcs(bucket,  text_file.replace("stt_synthetic_tests_data", "stt_synthetic_results").replace(".txt","") + f"-gemini_{split_strategy}_{model_id}.txt",
        #                   gemini_file, tags )

        # with open(local_file_txt, 'r') as f:
        #   ground_truth = f.read()
        # ground_truth = ground_truth.replace("\n", " ")

        # wer, semantic_textual_similarity = evaluate_data([result], [ground_truth])
        # if debug: print(f"Results:{wav_file}, WER: {wer}, semantic_textual_similarity: {semantic_textual_similarity}")

        # if system_instruction:
        #   prompt_log = "system_instruction:" + system_instruction + "\nprompt:" +prompt
        # else:
        #   prompt_log = prompt

        # data = {
        #   "input_file": wav_file,
        #   "ground_truth": ground_truth,
        #   "model_name": split_strategy+model_name,
        #   "prompt": prompt_log,
        #   "wer": wer,
        #   "processing_time": elapsed_time,
        #   "semantic_textual_similarity": semantic_textual_similarity,
        #   "generated_file": gemini_file,
        #   "generated_text": result
        # }
        
        # df = pd.DataFrame( data = [data], columns = ["input_file","ground_truth", "wer", "semantic_textual_similarity","generated_file","generated_text" , "model_name", "prompt", "processing_time"])

        # save_results_df_bq(df, table_id, truncate=False)

        i += 1


In [ ]:
%%time

models_dic = {
  "gemini-1.5-pro": gemini_stt,
  "gemini-1.5-pro-001": gemini_stt,
  "gemini-1.5-pro-002": gemini_stt,
  "gemini-1.5-flash-002": gemini_stt,
  "gemini-1.5-flash": gemini_stt,
  # Finetuned model
  #"projects/801452371447/locations/us-central1/endpoints/3103157164630343680": gemini_stt,
}

split_strategies_dic = {
  "gcs_max_token:"    : None,
  "split_by_silences:": get_audio_sequence_split_by_silences,
  "hard_split:"       : get_audio_sequence_hard_split,
  # "no_split:"         : get_one_full_sequence,  
}

process_transcriptions(prompt, split_strategies_dic, models_dic)

In [ ]:
BUCKET_NAME

In [ ]:
%%time

system_instruction = """<ai_role>
  You are an AI transcriptionist specializing in interviews.
  Your primary function is to convert spoken language from audio files into accurate, well-formatted text. Audio files are in french, transcriptions are in french.
  Ensure the transcribed text is clear and readable.
  Add ponctuation like comma, question mark, exclamation mark, etc.
  Ignore background audio.
</ai_role>
<answer_format>
Output full word only.
Do not generate any other text.
Do not truncate words.
</answer_format>
"""

prompt = """Generate a transcription in French of the audio, only extract speech and ignore background audio.
Transcribe spoken words.
"""

models_dic = {
  "gemini-1.5-pro": gemini_stt,
  "gemini-1.5-pro-001": gemini_stt,
  "gemini-1.5-pro-002": gemini_stt,
  "gemini-1.5-flash-002": gemini_stt,
  "gemini-1.5-flash": gemini_stt,
  # Finetuned model
  "projects/801452371447/locations/us-central1/endpoints/3103157164630343680": gemini_stt,
}

split_strategies_dic = {

  "no_split:"         : get_one_full_sequence,
  "gcs_max_token:"    : None,
  "split_by_silences:": get_audio_sequence_split_by_silences,
  "hard_split:"       : get_audio_sequence_hard_split,
}

process_transcriptions(prompt)